### Data Transformation

In [0]:
from pyspark.sql.functions import col, when, count, to_date,mean , avg, floor
from pyspark.sql.types import IntegerType, StringType


In [0]:
df = spark.read.format('csv')\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@storageforproject.dfs.core.windows.net/netflix_titles/*")

display(df)

In [0]:
# check the nulls in dataframe
df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).show()

In [0]:
df.printSchema()

In [0]:
# The _rescued_data column in your DataFrame is created by Databricks Auto Loader when it encounters data that does not conform to the expected schema.
df = df.drop('_rescued_data')

In [0]:
# change the datatype of duration_minutes from string to integer
df = df.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))

# check the average and mean of the duration_min column
duration_min_stats = df.select(mean("duration_minutes").alias("mean_duration"), avg("duration_minutes").alias("avg_duration"))
duration_min_stats.display()

# 
mean_duration = df.select(mean('duration_minutes')).collect()[0][0]
# fill na
df = df.fillna(mean_duration, subset=['duration_minutes'])

# display
display(df)


In [0]:
# check the uniqueness in the column
df.groupBy('duration_seasons').count().orderBy(col('count').desc()).display()
# change the value where greater than 13
df = df.withColumn("duration_seasons", when(col("duration_seasons") > 13, 13).otherwise(col("duration_seasons")))

# change the dataype of col
df= df.withColumn("duration_seasons",col("duration_seasons").cast(IntegerType()))

# fill the null value with mean
duration_seasons_mean = df.select(floor(mean("duration_seasons"))).collect()[0][0]
 
df= df.fillna(duration_seasons_mean,subset=["duration_seasons"])

display(df)

In [0]:
# get most frequent rating
most_freq_rating = df.groupBy("rating").count().orderBy(col("count").desc()).first()[0]

# fill na with the most frequent rating
df = df.fillna(most_freq_rating,subset=["rating"])

In [0]:
# get most frequent title and type
most_freq_type = df.groupBy("type").count().orderBy(col("count").desc()).first()[0]

# fill na with the most frequent title and type
df = df.fillna({"title": "No title", "type": most_freq_type})

In [0]:
# # change datype to int
# df = df.withColumn("show_id", col('show_id').cast(IntegerType()))
# most_frequent_released_yr = df.groupBy(col('release_year').cast(StringType())).count().orderBy(col('count').desc()).first()[0]

# # fill na 
# df = df.fillna({"show_id":1000, "description": "No description", "release_year":most_frequent_released_yr, "date_added":"01/01/2000"})

In [0]:
df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).show()

In [0]:
# write to delta table

df.write.format("delta")\
    .mode("overwrite")\
    .option("path", "abfss://silver@storageforproject.dfs.core.windows.net/netflix_titles")\
    .save()